#Engenharia Gramatical

Ano Letivo 22/23

Autor Tiago Baptista

# Aula 2

## Sumário



1.   O que é o lark ?
2.   Principais características
  - construções automáticas
  - ferramentas de visualização
  - suporte de ebnf
4.   lexer em lark
5.   Introdução aos transformers

## Notas
[lark cheatsheet](https://lark-parser.readthedocs.io/en/latest/_static/lark_cheatsheet.pdf)

[Documentação](https://lark-parser.readthedocs.io)

[lark common](https://github.com/lark-parser/lark/blob/master/lark/grammars/common.lark)


### Demo introdução a lark e Tranformers

####Exercício 1
Escreva uma gramática independente de contexto para definir uma lista possivelmente vazia de números inteiros separados por vírgulas.

Deve importar o módulo **Lark** para gerar um parser que ao reconhecer uma frase gere a respetiva árvore de parsing.

Use a função de *pretty print* do módulo *Lark* para imprimir a árvore construida.

Nesta primeira versão não se pretende que faça nenhuma travessia à árvore, 
mas em contrapartida pretende-se que escreva várias versões da GIC para usar extended-BNF (uso de operadores para descrever repetições) e para usar apenas BNF (uso de recursividade par descrever repetições). 




In [2]:
%pip install lark
from lark import Lark,Token
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard



grammar1 = '''
//Regras Sintaticas
start: PE elementos PD
elementos : 
          | elemento (VIR elemento)*
elemento : NUMERO 
//Regras Lexicográficas
NUMERO:"0".."9"+ // [0-9]+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''
grammar2 = '''
//Regras Sintaticas
start: PE elementos PD
elementos: elemento VIR elementos
        |
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar3 = '''
//Regras Sintaticas
start: PE elementos PD
elementos: elementos VIR elemento
        |
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar4 = '''
//Regras Sintaticas
start: PE ( |elementos) PD
elementos: elemento VIR elementos
//elementos: elemento VIR elementos
         | elemento
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar = '''
//Regras Sintaticas
start: PE elemento* PD
elemento : NUMERO (VIR NUMERO)*
//Regras Lexicográficas
NUMERO:"0".."9"+ 
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''
# Soma da lista, encontrar o maior da lista
class ExemploTransformer(Transformer):
  def start(self,elementos):
    print("start")
    
  def elemento(self,elemento):
    print("elemento soma")
    print(sum(elemento))
    
  def NUMERO (self,numero):
    print("numero")
    print(numero)
    return int(numero)
    
  def PE(self,pe):
    print("pe")
    print(pe)
    return str(pe)

  def PD(self,pd):
    print("pd")
    print(pd)
    return str(pd)

  def VIR(self,vir):
    print("vir")
    print(vir)
    return Discard

  pass

frase = "[1,23,345]"

p = Lark(grammar)   #não muito bem
#p = Lark(grammar1)  #recomendada
#p = Lark(grammar2)  #incorreta
#p = Lark(grammar3)  #incorreta
#p = Lark(grammar4)   #aceitável

tree = p.parse(frase)
print(tree.pretty())
#for element in tree.children:
  #print(element)


data = ExemploTransformer().transform(tree) # chamar o transformer para obter
#print(data)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
start
  [
  elemento
    1
    ,
    23
    ,
    345
  ]

pe
[
numero
1
vir
,
numero
23
vir
,
numero
345
elemento soma
369
pd
]
start


####Exercício 2
Retome uma das GIC escrita em extended-BNF para definir uma linguagem que permita escrever uma lista possívelmente vazia de números inteiros separados por vírgulas,
mas agora use o respetivo Transformer e escreva funções para visitar os nodos da árvore de parsing e escrever o conteúdo de cada uma.


In [3]:
%pip install lark
from lark import Lark
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

class MyTransformer(Transformer):
    lista = []
    def start(self, items):
        output={}
        output["elementos"] = self.lista
        soma = sum(self.lista)
        output["soma"] = soma
        print("raiz")
        return (output)

    def elemento(self, elemento):
        r = list(filter(lambda x: x!=',', elemento))
        for x in r:
          print(x)
        return r

    def VIR(self,op):
        print(str(op))
        return Discard

    def NUMERO(self,numero):
        print(str(numero))
        self.lista.append(int(numero))
        return int(numero)

    def PE(self,pe):
        print(str(pe))

    def PD(self,pd):
        print(str(pd))

## Primeiro precisamos da GIC
grammar = '''
start: PE elemento* PD
elemento : primeiro resto 
primeiro : NUMERO
resto : (VIR NUMERO)*
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","

%import common.WS
%ignore WS
'''

frase = "[1,2,3]"
p = Lark(grammar)

parse_tree = p.parse(frase)
print(parse_tree.pretty())

data = MyTransformer().transform(parse_tree)
print(data)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
start
  [
  elemento
    primeiro	1
    resto
      ,
      2
      ,
      3
  ]

[
1
,
2
,
3
Tree(Token('RULE', 'primeiro'), [1])
Tree(Token('RULE', 'resto'), [2, 3])
]
raiz
{'elementos': [1, 2, 3], 'soma': 6}


#### Exercício 3


Desenvolva uma GIC para definir uma linguagem que permita escrever listas
mistas de números e palavras, de tal forma que as 3 frases abaixo sejam frases
válidas dessa linguagem:

LISTA 1 .

Lista aaa .

Lista 1, 2, agora, 3, 4, fim, 7, 8.

Resolva as seguintes alíneas recorrendo ao uso de Transformer :

(a) Calcule, retorne e imprima quantos elementos existem numa lista.

(b) Identifique, retorne e imprima o elemento que mais se repete numa lista.

(c) Calcule, retorne e imprima a soma de todos os números que se encontrem entre as palavras
agora e fim de uma lista.

(d) Verifique se uma lista é válida de acordo com os seguintes parâmetros :

- Caso a palavra "agora" ocorra então a palavra "fim" também tem de aparecer posteriormente na lista e o contrário também.
- Entre "agora" e "fim" tem de aparecer pelo menos um número.


In [33]:
from lark import Lark,Token
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

grammar = '''
//Regras Sintaticas
start: LISTA elementos* "."
elementos : elemento ("," elemento)*
elemento : NUMERO
        | PALAVRA
//Regras Lexicográficas
LISTA:"LISTA"i
NUMERO: INT
PALAVRA: WORD 

%import common.WS
%import common.INT
%import common.WORD
%ignore WS
'''

class ExemploTransformer(Transformer):
    counter = 0
    lista = {}
    output = {"counter" : 0, "elems" : {}, "moda" : "", "soma" : 0}
    agora = 0
    fim = 0
    dif = 0
    soma = 0
    erro = 0
    entre_elems = 0
    last = ""

    def start(self,elementos):
        self.output["counter"] = self.counter
        
        max = (0,0)
        for elem in self.lista:
            if self.lista[elem] > max[1]:
                max = (elem,self.lista[elem])
    
        self.output["moda"] = max[0]
        self.output["elems"] = self.lista
        self.output["soma"] = self.soma - self.dif

        if self.last == "agora":
            self.erro = 1

        if self.erro == 1:
            self.output = "Lista inválida"
        return self.output
    
    def elemento(self,elemento):
        self.counter += 1
        return elemento
        
    
    def NUMERO (self,numero):
        numero = int(numero)
        if numero in self.lista:
            self.lista[numero] += 1
        else:
            self.lista[numero] = 1

        if self.agora == 1:
            self.soma += numero
            self.dif += numero
            self.entre_elems += numero     
        
        return numero

    
    def PALAVRA (self,palavra):
        palavra = str(palavra)

        if palavra in self.lista:
            self.lista[palavra] += 1
        else:
            self.lista[palavra] = 1

        if palavra == "agora":
            self.agora += 1
            self.last = "agora"
    
        if palavra == "fim":
            self.fim += 1
            self.dif = 0
            if self.agora == 0 or self.entre_elems == 0:
                self.erro = 1
            self.entre_elems = 0
            self.last = "fim"
            

        return palavra

    def LISTA(self,lista):
        return Discard
    
    pass

frase = "LISTA agora,1,2,agora,3,3,3,7,fim ."

p = Lark(grammar)


tree = p.parse(frase)


data = ExemploTransformer().transform(tree)
print(data)

{'counter': 9, 'elems': {'agora': 2, 1: 1, 2: 1, 3: 3, 7: 1, 'fim': 1}, 'moda': 3, 'soma': 3}
